In [1]:
import canalystii
import usb.core
import usb.util


In [2]:
#show all available devices
# print(usb.core.show_devices())
devices = usb.core.find(find_all=True)
for device in devices:
    print(device)

DEVICE ID 04d8:0053 on Bus 000 Address 004 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x200 USB 2.0
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :    0x8 (8 bytes)
 idVendor               : 0x04d8
 idProduct              : 0x0053
 bcdDevice              :  0x334 Device 3.34
 iManufacturer          :    0x1 Microchip Technology Inc.
 iProduct               :    0x3 Chuangxin Tech USBCAN/CANalyst-II
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 500 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x66 (102 bytes)
   bNumInterfaces       :    0x1
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0xc0 Self Powered
   

In [2]:

# find our device
dev = usb.core.find(idVendor=0x04D8, idProduct=0x0053)

# was it found?
if dev is None:
    raise ValueError('Device not found')


ValueError: Device not found

In [3]:
dev.__del__()

AttributeError: 'NoneType' object has no attribute '__del__'

In [9]:
# Connect to the Canalyst-II device
# Passing a bitrate to the constructor causes both channels to be initialized and started.
dev = canalystii.CanalystDevice(bitrate=500000)



USBError: [Errno None] b'libusb0-dll:err [claim_interface] could not claim interface 0, invalid configuration 0\n'

In [7]:
# The canalystii.Message class is a ctypes Structure, to minimize overhead
new_message = canalystii.Message(can_id=0x300,
                                 remote=False,
                                 extended=False,
                                 data_len=8,
                                 data=(0x01,0x02,0x03,0x04,0x05,0x06,0x07,0x08))
# Send one copy to channel 1
dev.send(1, new_message)
# Send 3 copies to channel 0
# (argument can be an instance of canalystii.Message or a list of instances)
dev.send(0, [new_message] * 3)

In [8]:
dev.flush_tx_buffer(0)
dev.flush_tx_buffer(1)

True

In [9]:
#read the data
data = dev.receive(0)
print(data)
data = dev.receive(1)
print(data)

[CanalystMessage ID=0x300 TS=0xa3219 Data=0102030405060708, CanalystMessage ID=0x300 TS=0x2486790 Data=0102030405060708]
[CanalystMessage ID=0x300 TS=0xa3223 Data=0102030405060708, CanalystMessage ID=0x300 TS=0xa3225 Data=0102030405060708, CanalystMessage ID=0x300 TS=0xa3228 Data=0102030405060708, CanalystMessage ID=0x300 TS=0x2486793 Data=0102030405060708, CanalystMessage ID=0x300 TS=0x2486796 Data=0102030405060708, CanalystMessage ID=0x300 TS=0x2486798 Data=0102030405060708]


In [47]:
import json
import ctypes

In [69]:

#convert data to 8 bit array
def convert_data(data):
    arr = []
    #every 8 bits
    for i in range(0, len(data), 8):
        #every 8 bits
        arr.append((ctypes.c_ubyte * 8)(*[ctypes.c_ubyte(ord(c)) for c in data[i:i+8]]))
    return arr

def reconstruct_data(data):
    data = [chr(i) for i in data]
    return ''.join(data)

def send_message(data : ctypes.c_ubyte):
    new_message = canalystii.Message(can_id=0x300,
                                 remote=False,
                                 extended=False,
                                 data_len=len(data),
                                 data=data)
    dev.send(1, new_message)





In [73]:
data = {
    'ccs18_temp': 0,
    'ccs18_co2': 0,
    'ccs18_tvoc': 0,
    'dht11_temp': 0,
    'dht11_humid': 0,
}



#show data as hex
d = convert_data(data)


In [87]:
#print value as char
for i in range(len(d)):
    for i in d[i]:
        print(chr(i), end='')

{"ccs18_temp": 0, "ccs18_co2": 0, "ccs18_tvoc": 0, "dht11_temp": 0, "dht11_humid": 0}   

In [86]:
for i in range(len(d)):
    print('0x', end='')
    for i in d[i]:
        print(hex(i).replace('0x', ''), end='')   
    print('')

0x7b2263637331385f
0x74656d70223a2030
0x2c20226363733138
0x5f636f32223a2030
0x2c20226363733138
0x5f74766f63223a20
0x302c202264687431
0x315f74656d70223a
0x20302c2022646874
0x31315f68756d6964
0x223a20307d000


In [103]:
#send data
for i in range(len(d)):
    send_message(d[i])


USBError: [Errno None] b'libusb0-dll:err [submit_async] submitting request failed, win error: The device does not recognize the command.\r\n'

In [100]:
data = []
while True:
    try:
        data_temp = dev.receive(1).data
        if data_temp != None or data_temp != []:
            data.append(data_temp)
            print(data_temp)
    except:
        break

In [102]:
print(data[0]())

IndexError: list index out of range

In [95]:
data = dev.receive(0)
print(data)

[]


In [ ]:



#send data over can bus
new_message = canalystii.Message(can_id=0x300,
                                 remote=False,
                                 extended=False,
                                 data_len =

In [ ]:
# Stop both channels (need to call start() again to resume capturing or send any messages)
dev.stop(0)
dev.stop(1)